In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.3. Sociodemographic Imputation/Input')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.3. Sociodemographic Imputation/Output')
cur_date = "013023"

library(readxl)
library(tidyverse)
library(reshape2)
library(missForest)

# reading in files
acs_df = data.frame(read_excel("ACS_Data_013023.xlsx", sheet = 2))

# making some of the cols numeric
acs_df[,4:45] = apply(acs_df[,4:45], 2, as.numeric)
wildfire_hazard_df = data.frame(read_excel("Wildfire_Hazard_010523.xlsx"))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Warning message in apply(acs_df[, 4:45], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:45], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:45], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:45], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:45], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:45], 2, as.numeric):
“NAs introduced by coe

In [2]:
head(acs_df)
head(wildfire_hazard_df)

,GEO_ID,Census_Tract,County,Sex_Ratio,Median_Age,Population_Under_5,Population_Under_18,Population_Over_18,Population_Over_65,Race_White,⋯,Some_College,College_Education,Employed,Unemployed,Employed_Male,Employed_Female,Full_Time_Work,Part_Time_Work,No_Work,Average_Income_Deficit
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,14000US37001020100,201.00,Alamance,87.2,37.3,9.5,21.9,78.1,15.1,61.0,⋯,21.1,0.9,91.61621,8.383791,94.24951,89.20607,43.04307,21.93988,35.01704,5016
2,14000US37001020200,202.00,Alamance,78.3,31.0,7.1,29.1,70.9,11.2,51.9,⋯,18.9,12.4,89.54774,10.452261,86.97436,92.01970,39.42970,23.56604,37.00426,4007
3,14000US37001020300,203.00,Alamance,77.8,40.7,7.3,27.1,72.9,18.9,45.2,⋯,22.6,4.6,93.78094,6.219059,92.69841,94.80990,41.09615,18.98616,39.91770,5370
4,14000US37001020400,204.00,Alamance,78.1,30.5,8.9,31.4,68.6,10.1,19.6,⋯,16.3,5.4,91.69985,8.300153,93.36493,90.64532,37.55153,28.40066,34.04782,7148
5,14000US37001020501,205.01,Alamance,92.7,35.1,5.5,23.3,76.7,15.6,76.1,⋯,3.5,1.5,91.14707,8.852927,96.10136,86.41860,45.86115,25.56742,28.57143,7403
6,14000US37001020502,205.02,Alamance,99.0,32.1,9.8,30.0,70.0,8.9,40.7,⋯,10.8,5.2,89.44282,10.557185,89.64942,89.15094,43.57167,22.82645,33.60188,5412


,GEO_ID,County,Census_Tract,Avg_ACRES,Wildfire_Hazard_Potential_Mean,Wildfire_Hazard_Potential_Quintile_Number,Wildfire_Hazard_Potential_Quintile_Percentage
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,14000US37141920201,Pender,9202.01,596842.6,715.5177,5,0-20%
2,14000US37141920401,Pender,9204.01,596842.6,715.5177,5,0-20%
3,14000US37141920402,Pender,9204.02,596842.6,715.5177,5,0-20%
4,14000US37141920403,Pender,9204.03,596842.6,715.5177,5,0-20%
5,14000US37141920601,Pender,9206.01,596842.6,715.5177,5,0-20%
6,14000US37141990100,Pender,9901.00,596842.6,715.5177,5,0-20%


In [3]:
# seeing if the geo ids are consistent in each file
length(unique(acs_df$GEO_ID))
length(unique(wildfire_hazard_df$GEO_ID))

[1] 2195

[1] 2189

In [4]:
# they're not so what geo ids aren't consistent?
`%notin%` <- Negate(`%in%`)
ids_df = acs_df %>%
    filter(GEO_ID %notin% unique(wildfire_hazard_df$GEO_ID)) %>%
    select(c("GEO_ID", "Census_Tract", "County")) 

# ids that aren't consistent
inconsistent_ids = ids_df$GEO_ID
inconsistent_ids 

[1] "14000US37031990200" "14000US37055990200" "14000US37095990100"
[4] "14000US37095990200" "14000US37101041202" "14000US37137990100"

In [5]:
acs_df %>%
    filter(GEO_ID %in% inconsistent_ids)
wildfire_hazard_df %>%
    filter(GEO_ID %in% inconsistent_ids)

GEO_ID,Census_Tract,County,Sex_Ratio,Median_Age,Population_Under_5,Population_Under_18,Population_Over_18,Population_Over_65,Race_White,⋯,Some_College,College_Education,Employed,Unemployed,Employed_Male,Employed_Female,Full_Time_Work,Part_Time_Work,No_Work,Average_Income_Deficit
<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
14000US37031990200,9902.00,Carteret,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
14000US37055990200,9902.00,Dare,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
14000US37095990100,9901.00,Hyde,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
14000US37095990200,9902.00,Hyde,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
14000US37101041202,412.02,Johnston,92.8,36.5,7.2,26.8,73.2,12.9,81.9,⋯,3.2,14.7,94.1938,5.806198,92.40081,96.70433,38.68631,26.22378,35.08991,6035
14000US37137990100,9901.00,Pamlico,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


GEO_ID,County,Census_Tract,Avg_ACRES,Wildfire_Hazard_Potential_Mean,Wildfire_Hazard_Potential_Quintile_Number,Wildfire_Hazard_Potential_Quintile_Percentage
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>


Almost all ACS data is missing for these GEO ids that aren't in the WHP df, so they'll be removed.

In [6]:
acs_whp_df = inner_join(acs_df, wildfire_hazard_df)

head(acs_whp_df)

Joining, by = c("GEO_ID", "Census_Tract", "County")


,GEO_ID,Census_Tract,County,Sex_Ratio,Median_Age,Population_Under_5,Population_Under_18,Population_Over_18,Population_Over_65,Race_White,⋯,Employed_Male,Employed_Female,Full_Time_Work,Part_Time_Work,No_Work,Average_Income_Deficit,Avg_ACRES,Wildfire_Hazard_Potential_Mean,Wildfire_Hazard_Potential_Quintile_Number,Wildfire_Hazard_Potential_Quintile_Percentage
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,14000US37001020100,201.00,Alamance,87.2,37.3,9.5,21.9,78.1,15.1,61.0,⋯,94.24951,89.20607,43.04307,21.93988,35.01704,5016,278233.8,92.23704,1,81-100
2,14000US37001020200,202.00,Alamance,78.3,31.0,7.1,29.1,70.9,11.2,51.9,⋯,86.97436,92.01970,39.42970,23.56604,37.00426,4007,278233.8,92.23704,1,81-100
3,14000US37001020300,203.00,Alamance,77.8,40.7,7.3,27.1,72.9,18.9,45.2,⋯,92.69841,94.80990,41.09615,18.98616,39.91770,5370,278233.8,92.23704,1,81-100
4,14000US37001020400,204.00,Alamance,78.1,30.5,8.9,31.4,68.6,10.1,19.6,⋯,93.36493,90.64532,37.55153,28.40066,34.04782,7148,278233.8,92.23704,1,81-100
5,14000US37001020501,205.01,Alamance,92.7,35.1,5.5,23.3,76.7,15.6,76.1,⋯,96.10136,86.41860,45.86115,25.56742,28.57143,7403,278233.8,92.23704,1,81-100
6,14000US37001020502,205.02,Alamance,99.0,32.1,9.8,30.0,70.0,8.9,40.7,⋯,89.64942,89.15094,43.57167,22.82645,33.60188,5412,278233.8,92.23704,1,81-100


# Variable & GEO Id Filtering

Removing variables and GEO ids that have less than 25% of data prior to imputation

In [7]:
# only keeping variables with at least 25% of data 
variable_presence_df = acs_whp_df[,1:45] %>% 
    pivot_longer(cols = 4:45, names_to = "Variable", values_to = "Value") %>%
    # creating 2 count cols: 1st for non-missing data, 2nd for all data
    mutate(data_count = ifelse(is.na(Value), 0, 1), all_count = 1) %>%
    group_by(Variable) %>%
    summarize(Variable_Presence_Percent = sum(data_count)/ sum(all_count) * 100) %>%
    arrange(Variable_Presence_Percent)

head(variable_presence_df)

Variable,Variable_Presence_Percent
<chr>,<dbl>
Poverty_Asian,71.90498
Average_Income_Deficit,78.80311
Poverty_Other_Race,79.85381
Poverty_Mixed_Race,92.91914
Poverty_Black,94.79214
Poverty_Hispanic_Latino,96.80219


In [8]:
# geo id percentages
geo_id_percent_occurence = apply(acs_whp_df[4:45], 1, function(x) (length(na.omit(x))/length(x)) * 100)

# creating a df and adding back in geo ids
id_presence_df = data.frame(matrix(ncol = 2, nrow = length(geo_id_percent_occurence)))
colnames(id_presence_df) = c("GEO_ID", "GEO_ID_Presence_Percentage")
id_presence_df$GEO_ID = acs_whp_df$GEO_ID
id_presence_df$GEO_ID_Presence_Percentage = geo_id_percent_occurence
                                 
id_presence_df = id_presence_df %>%
    filter(GEO_ID_Presence_Percentage < 25) %>%
    arrange(GEO_ID_Presence_Percentage)
                                 
head(id_presence_df)

,GEO_ID,GEO_ID_Presence_Percentage
,<chr>,<dbl>
1,14000US37019990100,4.761905
2,14000US37031980100,4.761905
3,14000US37031990100,4.761905
4,14000US37051980100,4.761905
5,14000US37053990100,4.761905
6,14000US37055990100,4.761905


In [9]:
# getting 20 geo ids to be removed
removed_geo_ids = id_presence_df$GEO_ID


filtered_acs_whp_df = acs_whp_df %>%
    filter(GEO_ID %notin% removed_geo_ids)

head(filtered_acs_whp_df)
dim(filtered_acs_whp_df)

,GEO_ID,Census_Tract,County,Sex_Ratio,Median_Age,Population_Under_5,Population_Under_18,Population_Over_18,Population_Over_65,Race_White,⋯,Employed_Male,Employed_Female,Full_Time_Work,Part_Time_Work,No_Work,Average_Income_Deficit,Avg_ACRES,Wildfire_Hazard_Potential_Mean,Wildfire_Hazard_Potential_Quintile_Number,Wildfire_Hazard_Potential_Quintile_Percentage
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,14000US37001020100,201.00,Alamance,87.2,37.3,9.5,21.9,78.1,15.1,61.0,⋯,94.24951,89.20607,43.04307,21.93988,35.01704,5016,278233.8,92.23704,1,81-100
2,14000US37001020200,202.00,Alamance,78.3,31.0,7.1,29.1,70.9,11.2,51.9,⋯,86.97436,92.01970,39.42970,23.56604,37.00426,4007,278233.8,92.23704,1,81-100
3,14000US37001020300,203.00,Alamance,77.8,40.7,7.3,27.1,72.9,18.9,45.2,⋯,92.69841,94.80990,41.09615,18.98616,39.91770,5370,278233.8,92.23704,1,81-100
4,14000US37001020400,204.00,Alamance,78.1,30.5,8.9,31.4,68.6,10.1,19.6,⋯,93.36493,90.64532,37.55153,28.40066,34.04782,7148,278233.8,92.23704,1,81-100
5,14000US37001020501,205.01,Alamance,92.7,35.1,5.5,23.3,76.7,15.6,76.1,⋯,96.10136,86.41860,45.86115,25.56742,28.57143,7403,278233.8,92.23704,1,81-100
6,14000US37001020502,205.02,Alamance,99.0,32.1,9.8,30.0,70.0,8.9,40.7,⋯,89.64942,89.15094,43.57167,22.82645,33.60188,5412,278233.8,92.23704,1,81-100


[1] 2169   49

All variables will be kept, but now we're left with 2,169 geo ids. 

# Random Forest Imputation
Generating missing values using random forest (RF).

In [10]:
# imputing sociodemographic data using RF
random_forest_imputation = function(preimputed_df){

    imputed_RF_object = missForest(as.matrix(preimputed_df[,4:45]))
    imputed_RF_df = imputed_RF_object$ximp
    imputed_values_df = data.frame(cbind(preimputed_df[,c(1:3,46:49)], imputed_RF_df)) 

    
    return(imputed_values_df)
}

# calling function
imputed_df = random_forest_imputation(acs_whp_df)
head(imputed_df)

,GEO_ID,Census_Tract,County,Avg_ACRES,Wildfire_Hazard_Potential_Mean,Wildfire_Hazard_Potential_Quintile_Number,Wildfire_Hazard_Potential_Quintile_Percentage,Sex_Ratio,Median_Age,Population_Under_5,⋯,Some_College,College_Education,Employed,Unemployed,Employed_Male,Employed_Female,Full_Time_Work,Part_Time_Work,No_Work,Average_Income_Deficit
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,14000US37001020100,201.00,Alamance,278233.8,92.23704,1,81-100,87.2,37.3,9.5,⋯,21.1,0.9,91.61621,8.383791,94.24951,89.20607,43.04307,21.93988,35.01704,5016
2,14000US37001020200,202.00,Alamance,278233.8,92.23704,1,81-100,78.3,31.0,7.1,⋯,18.9,12.4,89.54774,10.452261,86.97436,92.01970,39.42970,23.56604,37.00426,4007
3,14000US37001020300,203.00,Alamance,278233.8,92.23704,1,81-100,77.8,40.7,7.3,⋯,22.6,4.6,93.78094,6.219059,92.69841,94.80990,41.09615,18.98616,39.91770,5370
4,14000US37001020400,204.00,Alamance,278233.8,92.23704,1,81-100,78.1,30.5,8.9,⋯,16.3,5.4,91.69985,8.300153,93.36493,90.64532,37.55153,28.40066,34.04782,7148
5,14000US37001020501,205.01,Alamance,278233.8,92.23704,1,81-100,92.7,35.1,5.5,⋯,3.5,1.5,91.14707,8.852927,96.10136,86.41860,45.86115,25.56742,28.57143,7403
6,14000US37001020502,205.02,Alamance,278233.8,92.23704,1,81-100,99.0,32.1,9.8,⋯,10.8,5.2,89.44282,10.557185,89.64942,89.15094,43.57167,22.82645,33.60188,5412


In [11]:
# exporting data
write.csv(imputed_df, paste0(Output,"/", cur_date, "_Imputed_ACS_WHP_Data.csv"), row.names = FALSE)